# 퍼널 분석

### 사전 분석 : hits
- 사용자 행동에 대한 상세 정보가 담긴 hits 컬럼값 분석

In [1]:
import os
import ast
import json
import random
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
color = sns.color_palette()

%matplotlib inline

from plotly import tools
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go

from sklearn import model_selection, preprocessing, metrics
import lightgbm as lgb

pd.options.mode.chained_assignment = None
pd.options.display.max_columns = 999

from pandas import json_normalize

In [2]:
train_path = 'new_train.csv'
train_df = pd.read_csv(train_path)

/var/folders/l3/cq8y8n3s03v315zt58m2zrv80000gn/T/ipykernel_16706/521266735.py:2: DtypeWarning:

Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.



In [3]:
hits_data = ast.literal_eval(train_df['hits'][0])

hits_data[0]

{'hitNumber': '1',
 'time': '0',
 'hour': '10',
 'minute': '54',
 'isInteraction': True,
 'isEntrance': True,
 'page': {'pagePath': '/home',
  'hostname': 'shop.googlemerchandisestore.com',
  'pageTitle': 'Home',
  'pagePathLevel1': '/home',
  'pagePathLevel2': '',
  'pagePathLevel3': '',
  'pagePathLevel4': ''},
 'appInfo': {'screenName': 'shop.googlemerchandisestore.com/home',
  'landingScreenName': 'shop.googlemerchandisestore.com/home',
  'exitScreenName': 'shop.googlemerchandisestore.com/google+redesign/apparel/womens+tshirts/womens+hero+vneck+tee+white.axd',
  'screenDepth': '0'},
 'exceptionInfo': {'isFatal': True},
 'product': [],
 'promotion': [],
 'eCommerceAction': {'action_type': '0', 'step': '1'},
 'experiment': [],
 'customVariables': [],
 'customDimensions': [],
 'customMetrics': [],
 'type': 'PAGE',
 'social': {'socialNetwork': '(not set)',
  'hasSocialSourceReferral': 'No',
  'socialInteractionNetworkAction': ' : '},
 'contentGroup': {'contentGroup1': '(not set)',
  'c

#### (1) 이탈률(bounce rate) 분석을 위한 주요 지표
- 이탈률 : 한 페이지 방문 후 다른 상호작용 없이 사이트를 떠난 방문자의 비율

**주요 Key**
- `isInteraction` : `False`이면, 해당 히트가 사용자의 상호작용이 없는 것으로 간주되어 이탈로 처리
- `isEntrance` : `True`이면, 해당 히트가 세션의 첫 번째 페이지
- `page.pagePath` : 방문한 페이지의 경로
- `eCommerceAction.action_type` : 0이면 일반적인 페이지 조회, 1이면 제품 조회, 2 이상이면 특정 상호작용(구매, 장바구니 추가 등)을 의미
---
#### (2) 종료율 (exit rate) 분석을 위한 주요 지표
- 종료율 : 특정 페이지에서 세션이 종료된 비율

**주요 Key**
- `appInfo.exitScreenName` : 방문자가 마지막으로 본 페이지
- `page.pagePath` : 방문한 페이지 경로
- `isEntrace` : `True`이면, 해당 페이지가 세션의 첫 페이지
- `type` : `PAGE` 이므로 페이지뷰 이벤트인지 확인
---
#### (3) 퍼널 분석 (Funnel Analysis) 분석을 위한 주요 지표
- 퍼널 분석 : 사용자가 특정 목표(구매 완료)까지 진행하는 과정을 추적하는데 사용

**주요 Key**
- `eCommerceAction.action_type` : 사용자의 전자상거래 행동 (0 = 기본 조회, 1 = 제품조회, 2 = 장바구니 추가, 3 = 결제 진행, 4 = 구매 완료 등)
- `eCommerceAction.step` : 퍼널의 어느 단계인지 표시(step1 : 페이지 방문, step2 : 장바구니 추가, step3 : 결제 진행, step 4 : 주문 완료 등)
- `page.pagePath` : 사용자가 어떤 경로를 통해 이동하는지 확인
- `appInfo.screenName` : 모바일 앱 화면의 경우 어떤 스크린에서 이벤트가 발생했는지
- `prodcut` : 제품 상세정보 (사용자가 본 상품이 무엇인지)
- `promotion` " 사용자에게 노출된 프로모션이 있는지

## 이탈률, 종료율, 퍼널 분석을 위한 hits 컬럼 펼치기

In [ ]:
df = train_df.copy()

# `hits` 컬럼을 문자열에서 리스트 형태로 변환
df["hits"] = df["hits"].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)

In [ ]:
# `explode()`를 사용하여 리스트 내 JSON을 개별 행으로 변환
df_expanded = df.explode("hits")

# JSON 데이터 (각 행의 `hits`가 JSON 객체) → 개별 컬럼으로 변환
df_expanded = df_expanded.join(pd.json_normalize(df_expanded["hits"]))

# 불필요한 원본 `hits` 컬럼 삭제
df_expanded.drop(columns=["hits"], inplace=True)

### (1) 이탈률(bounce rate) 분석
- 이탈률 = 이탈수(totals.bounces) / 방문수(전체 행)

In [ ]:
# 필터링
df_filtered = df_expanded[
    (df_expanded["type"] == "PAGE") &  # 페이지 조회만 포함
    (df_expanded["hitNumber"] == 1)  # 첫 번째 히트만 포함 (Landing Page)
]

# 그룹화하여 이탈률 계산
df_result = (
    df_filtered.groupby("page.pagePath")
    .agg(views=("page.pagePath", "count"),  # 페이지 조회수 (COUNT(*))
         bounce_rate=("totals.bounces", lambda x: x.sum() / x.count()))  # 이탈률 계산
    .reset_index()
)

# 정렬 및 상위 10개 출력
df_result = df_result.sort_values(by="views", ascending=False).head(10)

df_result